# This file contains functions used for analysing Bseries data 

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as glob
import ruptures as rpt
from scipy.signal import butter,filtfilt
from scipy.misc import derivative
import scipy.integrate as sint
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis


%matplotlib notebook

In [2]:
"""
Read all Bseries from 2021 data into one big dataframe
Find the path to files in subfolders using os.listdir
"""
def read_B_series_subfolders(path):
    # create a list of file and sub directories 
    # names in the given directory 
    subfolders = os.listdir(path)
    file_list = []

    # Iterate over all the entries
    for names in subfolders:
        # Create full path
        fullPath = os.path.join(path, names)
        # If entry is a directory then get the list of files in this directory 
        temp_path = fullPath#for glob
        if os.path.isdir(fullPath):
            file_list = file_list + read_B_series_subfolders(fullPath)   
        else:
            txt_files = glob.glob(path + "/*.csv")#only read txt files
            if (txt_files != []):
                file_list.append(txt_files)
    return file_list

def read_Bseries_data(path):
    drifter_identifyer = path[85:-4]
    print(drifter_identifyer)
    #column_names = ['time', 'pressure1', 'temp1', 'pressure2', 'temp2',"pressure3","temp3", 'E_x', 'E_y', 'E_z', 'W_w','Q_y', 'Q_z', 'M_x', 'M_y', 'M_z', 'AX_earth', 'AY_earth', 'AZ_earth',"RX","RY","RZ","CSM","CSA","CSR","CSTOT"]
    #indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    df = pd.read_csv(path)
    df.drop(df[abs(df["PL [hPa]"]) > 1200].index, inplace = True)
    df.drop(df[abs(df["PC [hPa]"]) > 1200].index, inplace = True)
    df.drop(df[abs(df["PR [hPa]"]) > 1200].index, inplace = True)
    return df

"""Function taken from somehere on the internett"""
#https://medium.com/analytics-vidhya/how-to-filter-noise-with-a-low-pass-filter-python-885223e5e9b7
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs  # Nyquist Frequency
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def window_cpd(signal,model,width,jump,dim,sigma):
    # change point detection
    model =   model#"ar"
    n = np.shape(signal)[0]
  
    algo = rpt.Window(model=model, width = width, jump = jump).fit(signal)
    my_bkps = algo.predict(pen=np.log(n) * dim *sigma ** 2)
    
    #plt.show()
    return my_bkps

# generate signal
#15.07 2021 A drone was used to follow drifter M09 at 12:59. Filneme for this deployment: M090715104232.txt

def gen_signal(filename):
    """Read data and create signal to be used for changepoint detection"""
    ruptures_df = read_Bseries_data(filename)
    #ruptures_df_norm = read_Bseries_data(filename)

    signal = ruptures_df[["PL [hPa]","PC [hPa]","PR [hPa]","QX [-]","QY [-]","QZ [-]"]].to_numpy()
    
    #ruptures_df_norm = (ruptures_df_norm-ruptures_df_norm.min())/(ruptures_df_norm.max()-ruptures_df_norm.min())
    #signal_norm = ruptures_df_norm[["PL [hPa]","PC [hPa]","PR [hPa]"]].to_numpy()
    
    #Send data through low pass filter
    low_pass_signal = ruptures_df[["PL [hPa]","PC [hPa]","PR [hPa]","QX [-]","QY [-]","QZ [-]"]].to_numpy()

    #calculate velocity from acceleration
    start = 0
    end = len(ruptures_df["QX [-]"])
    time = time_array(start,end)
    
    v_x =sint.cumtrapz(ruptures_df["QX [-]"], time) 
    v_y =sint.cumtrapz(ruptures_df["QY [-]"], time) 
    v_z =sint.cumtrapz(ruptures_df["QZ [-]"], time) 
 
    v_tot = np.sqrt(v_x**2 + v_y**2 + v_z**2)
    
    # Filter requirements.
    T = time[-1]         # Sample Period
    fs = 100.0       # sample rate, Hz
    cutoff = 3      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
    order = 2       # sin wave can be approx represented as quadratic
    n = int(T * fs) # total number of samples
    
    for i in range(signal.shape[1]):
        y = butter_lowpass_filter(low_pass_signal[:,i], cutoff, fs, order)
        low_pass_signal[:,i] = y
    
    sigma = signal.std()
    n = len(signal)
    
    
    
    return signal,low_pass_signal,v_tot


"""Create a time array from B series indexes"""
def time_array(start,end):
    time_s = np.linspace(0,end-start,end-start)/100#convert to seconds
    return time_s

"""find index of time in video of a given drifter deployment"""
def index_finder(cp_times,time_s,start_n):
    length = len(cp_times)
    indexes = np.zeros(length)
    for i in range(length):#find indexes in time_s of the features
        indexes[i] = (next(j for j, _ in enumerate(time_s) if np.isclose(_, cp_times[i], 0.01)))    
    indexes = indexes.astype(int) + start_n
    return indexes

def cross_section_comp(filepath,name,model,width,jump,dim,sigma,params,data_vis="off",bkps_vis="off",k=0):
    #Function used by Cross_section_comparison.ipynb 
    #parameter k just to seperate data in plot

    data, data_low_pass,v_tot = gen_signal(filepath)
    
    data_lp_mean = (data_low_pass[:,0]+data_low_pass[:,2])/2
    lp_mean_norm = np.linalg.norm(data_lp_mean)
    data_lp_mean = data_lp_mean#/lp_mean_norm
    
    acc_abs = np.sqrt(data_low_pass[:,3]**2 + data_low_pass[:,4]**2 + data_low_pass[:,5]**2)
    
    data_bkps = window_cpd(data[:,params],model,width,jump,dim,sigma)
    print(name,"- Number of detected cp: ",len(data_bkps))
    """if bkps_vis == "on":
        #show ruptures breakpoints
        rpt.show.display(data[:,params], data_bkps, figsize=(10, 6))
    #plt.plot(data_low_pass[:,params[0]][data_bkps[0]:])
    if data_vis =="on":
        start = data_bkps[0]
        print("start",start,name)
        end = data_bkps[-1]
        #plt.plot(time_array(start,end),data_lp_mean[start:end]+k)#plot wit seconds on x axis
        plt.plot(data_lp_mean[start:end]+k,label = name)#plot without seconds
        plt.ylabel('Pressure []')
        #plt.plot(time_array(start,end),acc_abs[start:end]+k)
        plt.title(name)"""
    return data, data_low_pass,data_bkps,data_lp_mean




def dtw_drifters(s1,s2):
    """Dynamic time warping"""
    path = dtw.warping_path(s1, s2)
    dtwvis.plot_warping(s1, s2, path)
    distance = dtw.distance(s1, s2)
